In [1]:
import xarray as xr
import numpy as np
import glob
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
import warnings
warnings.filterwarnings("ignore")
#from tqdm import tqdm

 ### Dataset preparation for Juelich, Lindenberg, Warsar, Vienna, Bourges, Madrid

## recap of data structure

In [12]:
data = xr.open_dataset('/p/project1/exaww/chatterjee1/dataset/iconeu/04/01/'+'iefrf00170000.nc')
date = data.time.dt.strftime('%Y-%m-%d').values[0]
timestamp = data.time.dt.strftime('%H:%M:%S').values[0]

In [4]:
data

<xarray.Dataset> Size: 7MB
Dimensions:              (time: 1, lon: 1377, lat: 657)
Coordinates:
  * time                 (time) datetime64[ns] 8B 2023-04-01T17:00:00
  * lon                  (lon) float64 11kB -23.5 -23.44 -23.38 ... 62.44 62.5
  * lat                  (lat) float64 5kB 29.5 29.56 29.62 ... 70.38 70.44 70.5
Data variables:
    SYNMSG_BT_CL_IR10.8  (time, lat, lon) float32 4MB ...
    SYNMSG_BT_CL_WV6.2   (time, lat, lon) float32 4MB ...
Attributes:
    CDI:          Climate Data Interface version 2.0.4 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    institution:  Deutscher Wetterdienst
    history:      Mon Nov 11 14:17:08 2024: cdo -f nc copy ./04/01/iefrf00170...
    CDO:          Climate Data Operators version 2.0.4 (https://mpimet.mpg.de...

In [7]:
data['SYNMSG_BT_CL_IR10.8'].shape

(1, 657, 1377)

In [8]:
data['lon'].shape

(1377,)

In [9]:
data['lon'].values

array([-23.5   , -23.4375, -23.375 , ...,  62.375 ,  62.4375,  62.5   ])

In [10]:
data['lat'].values

array([29.5   , 29.5625, 29.625 , 29.6875, 29.75  , 29.8125, 29.875 ,
       29.9375, 30.    , 30.0625, 30.125 , 30.1875, 30.25  , 30.3125,
       30.375 , 30.4375, 30.5   , 30.5625, 30.625 , 30.6875, 30.75  ,
       30.8125, 30.875 , 30.9375, 31.    , 31.0625, 31.125 , 31.1875,
       31.25  , 31.3125, 31.375 , 31.4375, 31.5   , 31.5625, 31.625 ,
       31.6875, 31.75  , 31.8125, 31.875 , 31.9375, 32.    , 32.0625,
       32.125 , 32.1875, 32.25  , 32.3125, 32.375 , 32.4375, 32.5   ,
       32.5625, 32.625 , 32.6875, 32.75  , 32.8125, 32.875 , 32.9375,
       33.    , 33.0625, 33.125 , 33.1875, 33.25  , 33.3125, 33.375 ,
       33.4375, 33.5   , 33.5625, 33.625 , 33.6875, 33.75  , 33.8125,
       33.875 , 33.9375, 34.    , 34.0625, 34.125 , 34.1875, 34.25  ,
       34.3125, 34.375 , 34.4375, 34.5   , 34.5625, 34.625 , 34.6875,
       34.75  , 34.8125, 34.875 , 34.9375, 35.    , 35.0625, 35.125 ,
       35.1875, 35.25  , 35.3125, 35.375 , 35.4375, 35.5   , 35.5625,
       35.625 , 35.6

In [13]:
lat_check = data['lat'][288:377]
lat_check.min()

<xarray.DataArray 'lat' ()> Size: 8B
array(47.5)

In [2]:
nc_file_loc = '/p/project1/exaww/chatterjee1/dataset/iconeu/'
output_file = "msgobs_108_juelichcrops_icon.nc"

months = {
    4:'04/',
    5:'05/',
    6:'06/',
    7:'07/',
    8:'08/',
    9:'09/',
}

log_file = nc_file_loc + "processed_files_log_juelich.txt"
nan_crop_file = nc_file_loc + "nan_files_log_juelich.txt"

In [15]:
sample_counter = 0  # Initialize a counter for unique sample naming

all_crops = []  # List to store all crops
all_lats = []   # List to store all latitude coordinates
all_lons = []   # List to store all longitude coordinates
all_times = []  # List to store all timestamps

target_lat = 51.0504 #aurillac = 44.9332 #nuremberg = 49.4543 #cabauw = 51.9653 #Sirta = 48.717 # zargosa = 41.6474 #madrid = 40.4167 # Bourges = 47.0812 # Vienna = 48.2081 # warsaw = 52.229 # lindenberg = 52.210 #juelich = 50.9224
target_lon = 13.7373 #aurillac = 2.4483 #nuremberg = 11.0746 #cabauw = 4.8979 #Sirta = 2.208 # Zargosa = -0.8861 # Madrid = -3.7033 # Bourges = 2.3980 # Vienna = 16.3713 # warsaw = 21.012 # lindenberg = 14.122 #Juelich = 6.3639
crop_size = 78
half_crop = crop_size // 2

first_write = True  # Flag to check if it's the first time writing to the file

# Iterate over each month
for _, key in enumerate(months.keys()):
    month_loc = nc_file_loc + months[key]
    day_folders = sorted(glob.glob(month_loc + '*/'))  # Finds all day subfolders within the month folder

    # Iterate over each day folder within the current month
    for day_folder in day_folders:
        nc_filepattern = "iefrf*.nc"
        nc_files = sorted(glob.glob(day_folder + nc_filepattern)) # '/p/project1/exaww/chatterjee1/dataset/iconeu/09/01/'

        for i, file in enumerate(nc_files):

            # Log the name of the current file
            with open(log_file, 'a') as log:
                log.write(f"{file}\n")

            data = xr.open_dataset(file)
            date = data.time.dt.strftime('%Y-%m-%d').values[0]
            timestamp = data.time.dt.strftime('%H:%M:%S').values[0]
            

            lat = data['lat'].values  # shape: (H,)
            lon = data['lon'].values  # shape: (W,)            
            bt_data = data['SYNMSG_BT_CL_IR10.8'][0, :, :].values  # shape: (H, W)

            # Find closest lat/lon indices
            lat_idx = np.abs(lat - target_lat).argmin()
            lon_idx = np.abs(lon - target_lon).argmin()
            
            # Define crop boundaries (clamp to array bounds)
            start_y = max(0, lat_idx - half_crop)
            end_y = start_y + crop_size
            start_x = max(0, lon_idx - half_crop)
            end_x = start_x + crop_size
            
            # Skip if crop goes beyond data bounds
            if end_y > bt_data.shape[0] or end_x > bt_data.shape[1]:
                continue
            
            # Crop the data
            crop = bt_data[start_y:end_y, start_x:end_x]

            # Skip if NaNs are present
            if np.isnan(crop).any():
                continue

            all_crops.append(crop)
            all_lats.append(lat[start_y:end_y])
            all_lons.append(lon[start_x:end_x])
            #all_times.append(timestamp)
            all_times.append(data.time.values[0])
    
            # Increment the sample counter
            sample_counter += 1
    
all_crops_np = np.array(all_crops)
all_lats_np = np.array(all_lats)
all_lons_np = np.array(all_lons)
all_times_np = np.array(all_times)





In [16]:
all_crops_np.shape, all_times_np.shape

((4575, 78, 78), (4575,))

In [16]:
data.time.values[0]

numpy.datetime64('2023-10-01T00:00:00.000000000')

In [17]:
output_file = "msgobs_108_dresdencrops_icon.nc"

# Create a dataset with the combined data
ds = xr.Dataset(
    {
        "model_dresden_108": (["sample", "y", "x"], all_crops_np)  # Data variable
    },
    coords={
        "sample": (["sample"], np.arange(len(all_crops_np))),  # Sample numbers
        "time": (["sample"], all_times_np),
        "lat": (["sample", "y"], all_lats_np),
        "lon": (["sample", "x"], all_lons_np)
    }
)


ds.to_netcdf('/p/scratch/exaww/chatterjee1/nn_obs/continuous/' + output_file, mode='w')


In [18]:
data_madrid = xr.open_dataset('/p/project1/exaww/chatterjee1/dataset/' + output_file)
data_madrid

<xarray.Dataset> Size: 117MB
Dimensions:           (sample: 4575, y: 78, x: 78)
Coordinates:
  * sample            (sample) int64 37kB 0 1 2 3 4 ... 4570 4571 4572 4573 4574
    time              (sample) <U8 146kB ...
    lat               (sample, y) float64 3MB ...
    lon               (sample, x) float64 3MB ...
Dimensions without coordinates: y, x
Data variables:
    model_madrid_108  (sample, y, x) float32 111MB ...

In [19]:
data_madrid.model_madrid_108.shape

(4575, 78, 78)

In [8]:
all_crops_np.shape

(4575, 78, 78)

In [14]:
sample_counter

4575

In [5]:
all_lats_np.shape

(4575, 78)

In [17]:
all_lats_np[:,:] == all_lats_np[:,:] 

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [18]:
all_lats_np[0,:].shape

(78,)

## NC to hdf5

In [18]:
def convert_nc_to_hdf5(nc_file, hdf5_file):
    # Open the NetCDF file using xarray
    ds = xr.open_dataset(nc_file)

    # Create the HDF5 file
    with h5py.File(hdf5_file, 'w') as hdf5_data:
        # Iterate over all variables in the xarray dataset
        for var_name in ds.data_vars:
            var_data = ds[var_name].values
            
            # Create a dataset in the HDF5 file
            hdf5_dataset = hdf5_data.create_dataset(
                var_name, 
                data=var_data, 
                dtype=var_data.dtype, 
                #chunks=chunking,  # Enable chunking if requested
                #compression=compression  # Apply compression if provided
            )

            # Copy variable attributes of the variable to the HDF5 dataset
            for attr_name, attr_value in ds[var_name].attrs.items():
                hdf5_dataset.attrs[attr_name] = attr_value
        
        # Iterate over all coordinates in the xarray dataset
        for coord_name in ds.coords:
            coord_data = ds[coord_name].values
            
            # Handle datetime64 directly
            if np.issubdtype(coord_data.dtype, np.datetime64):
                # Convert to ISO string, then to bytes
                coord_data = ds[coord_name].dt.strftime('%Y-%m-%dT%H:%M:%S').values.astype('S')
            
            elif coord_data.dtype == 'O':
                coord_data = coord_data.astype('S')
            
            hdf5_coord = hdf5_data.create_dataset(
                coord_name, 
                data=coord_data
            )
            
            # Copy coordinate attributes to the HDF5 dataset
            for attr_name, attr_value in ds[coord_name].attrs.items():
                hdf5_coord.attrs[attr_name] = attr_value
        
        # Copy global attributes
        for attr_name, attr_value in ds.attrs.items():
            hdf5_data.attrs[attr_name] = attr_value
    # Close the xarray dataset
    ds.close()


nc_file = '/p/scratch/exaww/chatterjee1/nn_obs/continuous/msgobs_108_dresdencrops_icon.nc'
hdf5_file = '/p/project1/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_108_dresdencrops_icon.h5'
convert_nc_to_hdf5(
    nc_file, 
    hdf5_file, 
    #chunking=False, 
    #compression=None
)
print(f"Converted {nc_file} to {hdf5_file}")

Converted /p/scratch/exaww/chatterjee1/nn_obs/continuous/msgobs_108_dresdencrops_icon.nc to /p/project1/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_108_dresdencrops_icon.h5


 ## Collection the WV 6.2 and IR 10.8 characterizing the clusters and transition

In [3]:
# File paths
nc_file_loc = '/p/project1/exaww/chatterjee1/dataset/iconeu/'
log_file = nc_file_loc + "processed_files_log.txt"
nan_crop_file = nc_file_loc + "nan_files_log.txt"

output_dir = "/p/project/exaww/chatterjee1/dataset/warmworld_datasets/"
os.makedirs(output_dir, exist_ok=True)

# Months
months = {
    4: '04/', 5: '05/', 6: '06/', 7: '07/', 8: '08/', 9: '09/',
}

# Channels and crop
channel_keys = {
    "WV": "SYNMSG_BT_CL_WV6.2",
    "IR": "SYNMSG_BT_CL_IR10.8"
}
crop_size = 78
half = crop_size // 2

# Site locations
sites = {
    #"juelich": (50.9224, 6.3639),
    #"lindenberg": (52.210, 14.122),
    #"warsaw": (52.229, 21.012),
    #"vienna": (48.2081, 16.3713),
    #"bourges": (47.0812, 2.3980),
    #"zaragoza": (41.6474, -0.8861),
    #"sirta": (48.717, 2.208),
    #"cabauw": (51.9653, 4.8979),
    "nuremberg": (49.4543, 11.0746),
    "aurillac": (44.9332, 2.4483),
    "dresden": (51.0504, 13.7373),
}

# Storage dictionaries
all_crops = {site: {ch: [] for ch in channel_keys} for site in sites}
all_lats = {site: [] for site in sites}
all_lons = {site: [] for site in sites}
all_times = {site: [] for site in sites}

# Utility
def get_crop_center_indices_1d(lat_1d, lon_1d, lat_pt, lon_pt):
    iy = np.abs(lat_1d - lat_pt).argmin()
    ix = np.abs(lon_1d - lon_pt).argmin()
    return iy, ix

# Processing
for month_key, month_str in months.items():
    month_loc = os.path.join(nc_file_loc, month_str)
    day_folders = sorted(glob.glob(month_loc + '*/'))

    for day_folder in day_folders:
        nc_files = sorted(glob.glob(os.path.join(day_folder, "iefrf*.nc")))

        #for fpath in tqdm(nc_files, desc=f"Processing {month_str}"):
        for fpath in nc_files:
            try:
                ds = xr.open_dataset(fpath)
                lat_grid = ds['lat'].values
                lon_grid = ds['lon'].values

                for site_name, (site_lat, site_lon) in sites.items():
                    iy, ix = get_crop_center_indices_1d(lat_grid, lon_grid, site_lat, site_lon)

                    if (iy - half < 0 or iy + half > lat_grid.shape[0] or
                        ix - half < 0 or ix + half > lon_grid.shape[0]):
                        print(f"Skipping {fpath} for {site_name}: crop out of bounds")
                        continue

                    # Collect crops
                    has_nan = False
                    for ch_name, ch_key in channel_keys.items():
                        bt_data = ds[ch_key][0, :, :].values
                        bt_crop = bt_data[iy - half: iy + half, ix - half: ix + half]

                        if bt_crop.shape != (crop_size, crop_size):
                            has_nan = True
                            break

                        if np.isnan(bt_crop).any():
                            has_nan = True
                        #   break

                        all_crops[site_name][ch_name].append(bt_crop)

                    if has_nan:
                        with open(nan_crop_file, 'a') as log_nan:
                            log_nan.write(f"NaN or invalid crop in {fpath} for {site_name}\n")
                        continue

                    lat_crop = lat_grid[iy - half: iy + half]
                    lon_crop = lon_grid[ix - half: ix + half]
                    lat_2d, lon_2d = np.meshgrid(lat_crop, lon_crop, indexing='ij')

                    all_lats[site_name].append(lat_2d)
                    all_lons[site_name].append(lon_2d)
                    all_times[site_name].append(ds.time.values[0])

                with open(log_file, 'a') as log:
                    log.write(f"{fpath}\n")

            except Exception as e:
                print(f"Failed to process {fpath}: {e}")
            finally:
                ds.close()
    
# Save per-site NetCDF
for site in sites:
    print(f"Writing data for site: {site}")
    crops_np = {ch: np.array(all_crops[site][ch]) for ch in channel_keys}
    lats_np = np.array(all_lats[site])
    lons_np = np.array(all_lons[site])
    times_np = np.array(all_times[site])

    num_samples = times_np.shape[0]

    ds_out = xr.Dataset(
        {
            f"{site}_data_{ch}": (["sample", "y", "x"], crops_np[ch])
            for ch in channel_keys
        },
        coords={
            "sample": np.arange(num_samples),
            "lat": (["sample", "y", "x"], lats_np),
            "lon": (["sample", "y", "x"], lons_np),
            "time": (["sample"], times_np)
        }
    )

    out_path = os.path.join(output_dir, f"icon_{site}_WV_IR_crops.nc")
    ds_out.to_netcdf(out_path, mode="w")
    print(f"Saved: {out_path}")

Writing data for site: nuremberg
Saved: /p/project/exaww/chatterjee1/dataset/warmworld_datasets/icon_nuremberg_WV_IR_crops.nc
Writing data for site: aurillac
Saved: /p/project/exaww/chatterjee1/dataset/warmworld_datasets/icon_aurillac_WV_IR_crops.nc
Writing data for site: dresden
Saved: /p/project/exaww/chatterjee1/dataset/warmworld_datasets/icon_dresden_WV_IR_crops.nc


In [4]:
# File paths
nc_file_loc = '/p/project1/exaww/chatterjee1/dataset/iconeu/'
log_file = nc_file_loc + "processed_files_log.txt"
nan_crop_file = nc_file_loc + "nan_files_log.txt"

output_dir = "/p/project/exaww/chatterjee1/dataset/"
os.makedirs(output_dir, exist_ok=True)

# Months
months = {
    4: '04/', 5: '05/', 6: '06/', 7: '07/', 8: '08/', 9: '09/',
}

# Channels and crop
channel_keys = {
    "WV": "SYNMSG_BT_CL_WV6.2",
    "IR": "SYNMSG_BT_CL_IR10.8"
}
crop_size = 78
half = crop_size // 2

# Site locations
sites = {
    #"juelich": (50.9224, 6.3639),
    #"lindenberg": (52.210, 14.122),
    #"warsaw": (52.229, 21.012),
    #"vienna": (48.2081, 16.3713),
    "bourges": (47.0812, 2.3980),
    "zaragoza": (41.6474, -0.8861),
    "sirta": (48.717, 2.208),
    "cabauw": (51.9653, 4.8979),
}

# Storage dictionaries
all_crops = {site: {ch: [] for ch in channel_keys} for site in sites}
all_lats = {site: [] for site in sites}
all_lons = {site: [] for site in sites}
all_times = {site: [] for site in sites}

# Utility
def get_crop_center_indices_1d(lat_1d, lon_1d, lat_pt, lon_pt):
    iy = np.abs(lat_1d - lat_pt).argmin()
    ix = np.abs(lon_1d - lon_pt).argmin()
    return iy, ix

# Processing
for month_key, month_str in months.items():
    month_loc = os.path.join(nc_file_loc, month_str)
    day_folders = sorted(glob.glob(month_loc + '*/'))

    for day_folder in day_folders:
        nc_files = sorted(glob.glob(os.path.join(day_folder, "iefrf*.nc")))

        #for fpath in tqdm(nc_files, desc=f"Processing {month_str}"):
        for fpath in nc_files:
            try:
                ds = xr.open_dataset(fpath)
                lat_grid = ds['lat'].values
                lon_grid = ds['lon'].values

                for site_name, (site_lat, site_lon) in sites.items():
                    iy, ix = get_crop_center_indices_1d(lat_grid, lon_grid, site_lat, site_lon)

                    if (iy - half < 0 or iy + half > lat_grid.shape[0] or
                        ix - half < 0 or ix + half > lon_grid.shape[0]):
                        print(f"Skipping {fpath} for {site_name}: crop out of bounds")
                        continue

                    # Collect crops
                    has_nan = False
                    for ch_name, ch_key in channel_keys.items():
                        bt_data = ds[ch_key][0, :, :].values
                        bt_crop = bt_data[iy - half: iy + half, ix - half: ix + half]

                        if bt_crop.shape != (crop_size, crop_size):
                            has_nan = True
                            break

                        if np.isnan(bt_crop).any():
                            has_nan = True
                        #   break

                        all_crops[site_name][ch_name].append(bt_crop)

                    if has_nan:
                        with open(nan_crop_file, 'a') as log_nan:
                            log_nan.write(f"NaN or invalid crop in {fpath} for {site_name}\n")
                        continue

                    lat_crop = lat_grid[iy - half: iy + half]
                    lon_crop = lon_grid[ix - half: ix + half]
                    lat_2d, lon_2d = np.meshgrid(lat_crop, lon_crop, indexing='ij')

                    all_lats[site_name].append(lat_2d)
                    all_lons[site_name].append(lon_2d)
                    all_times[site_name].append(ds.time.values[0])

                with open(log_file, 'a') as log:
                    log.write(f"{fpath}\n")

            except Exception as e:
                print(f"Failed to process {fpath}: {e}")
            finally:
                ds.close()
    
# Save per-site NetCDF
for site in sites:
    print(f"Writing data for site: {site}")
    crops_np = {ch: np.array(all_crops[site][ch]) for ch in channel_keys}
    lats_np = np.array(all_lats[site])
    lons_np = np.array(all_lons[site])
    times_np = np.array(all_times[site])

    num_samples = times_np.shape[0]

    ds_out = xr.Dataset(
        {
            f"{site}_data_{ch}": (["sample", "y", "x"], crops_np[ch])
            for ch in channel_keys
        },
        coords={
            "sample": np.arange(num_samples),
            "lat": (["sample", "y", "x"], lats_np),
            "lon": (["sample", "y", "x"], lons_np),
            "time": (["sample"], times_np)
        }
    )

    out_path = os.path.join(output_dir, f"icon_{site}_WV_IR_crops.nc")
    ds_out.to_netcdf(out_path, mode="w")
    print(f"Saved: {out_path}")

Writing data for site: bourges
Saved: /p/project/exaww/chatterjee1/dataset/icon_bourges_WV_IR_crops.nc
Writing data for site: zaragoza
Saved: /p/project/exaww/chatterjee1/dataset/icon_zaragoza_WV_IR_crops.nc
Writing data for site: sirta
Saved: /p/project/exaww/chatterjee1/dataset/icon_sirta_WV_IR_crops.nc
Writing data for site: cabauw
Saved: /p/project/exaww/chatterjee1/dataset/icon_cabauw_WV_IR_crops.nc


In [19]:
bt_crop

array([[275.43866, 278.03827, 277.63593, ..., 268.01483, 267.71405,
        269.46796],
       [278.01483, 277.91718, 277.8742 , ..., 266.1906 , 265.2707 ,
        266.46405],
       [275.39374, 276.89764, 278.26093, ..., 266.72382, 264.0871 ,
        262.425  ],
       ...,
       [281.14178, 280.966  , 281.4699 , ..., 283.47772, 283.47382,
        283.4875 ],
       [281.6906 , 281.6867 , 281.75702, ..., 283.4367 , 283.43085,
        283.4367 ],
       [281.63202, 281.6867 , 281.78436, ..., 283.45038, 283.44257,
        283.42108]], dtype=float32)

In [23]:
crops_np

{'WV': array([], dtype=float64), 'IR': array([], dtype=float64)}

In [13]:
crops_np

{'WV': array([], dtype=float64), 'IR': array([], dtype=float64)}

In [14]:
lat_crop.shape

NameError: name 'lat_crop' is not defined

In [24]:
all_crops

{'juelich': {'WV': [], 'IR': []},
 'lindenberg': {'WV': [], 'IR': []},
 'warsaw': {'WV': [], 'IR': []},
 'vienna': {'WV': [], 'IR': []},
 'bourges': {'WV': [], 'IR': []},
 'zaragoza': {'WV': [], 'IR': []},
 'sirta': {'WV': [], 'IR': []},
 'cabauw': {'WV': [], 'IR': []}}